In [2]:
import pandas as pd
import numpy as np
import scipy as sp

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import decomposition
from sklearn import datasets

In [4]:
shows2 = pd.read_csv('~/desktop/clusters_with_labels.csv')
shows2 = shows2.iloc[:,1:]
shows2.head()

,sdid,EP000000211718,EP000000211720,EP000000351287,EP000006528880,EP000010050032,EP000010050038,EP000011894177,EP000011894180,EP000014577585,...,SH023744650000,SH023792650000,SH023849750000,SH023889310000,SH023939730000,SH023976860000,SH023997410000,SH024008270000,SH024008910000,cluster
0,\x00004f6dddb126b5c3e50bde7fdc4a4562fa2b1c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,\x00035ff6989ae36256a790079bf97dc062d48d00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,\x000362901917b4c0ecd6340ce9fa1ac7a90934e5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,0,0,3
3,\x000434aae5717a2ac4a6707243308ef247ff8e10,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,\x0004bd646e69d2e60e369903a02e8ea9df6cb2e4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [5]:
data = pd.read_csv('~/desktop/clustering_table.csv')
data.head()

,sdid,show1,show2,show3,genre1,genre2,callsign1,callsign2,callsign3,daypart1,daypart2,dma
0,\x007f4ee8b45fad0a4774d1d60846ed144531577b,EP00316978,EP00521262,EP01589925,Crime Drama,Reality,USAHD,HSTRYHD,WNBCDT,daytime,earlymorning,618
1,\x5d758c772f763bf16b0540f0e4c198cad7f16834,EP00497113,EP00692873,EP02332922,Talk,Entertainment,EHD,ESPNHD,NaN,primeaccess,primetime,547
2,\x8413c4198aebcc6d414d80f3e02c42ab19eefc9b,EP01324002,EP02183818,EP01789967,Reality,Drama,FOODHD,AMCHD,WABCDT,primetime,daytime,504
3,\xe4a9aece9b192950b1ffcdc359734ead616ff5ee,EP01567915,SH00591081,SH00925021,Sports Talk,Sports Related,ESPNHD,ESPNWHD,NFLHD,primetime,earlyfringe,501
4,\x8ee341136f3991447aecfb750ef279567fd98a98,EP01253151,EP01021086,EP01883603,Documentary,Reality,IDHD,NGCHD,SCIHD,primetime,earlymorning,623


In [25]:
analyze_me = pd.merge(data, shows2, how = 'inner', on='sdid')
analyze_me.head()

,sdid,show1,show2,show3,genre1,genre2,callsign1,callsign2,callsign3,daypart1,...,SH023744650000,SH023792650000,SH023849750000,SH023889310000,SH023939730000,SH023976860000,SH023997410000,SH024008270000,SH024008910000,cluster
0,\x28d8badcfd0a79edcee94962ee23fbfa1e09a589,SH00019917,EP02215786,SH01092572,Sports Related,Reality,ESPNHD,NFLHD,NBATV,latefringe,...,0,0,0,0,0,0,0,0,0,0
1,\x05a58f5e3d66fa67c1d713ac6826522b0f059d19,EP01532516,EP01333672,EP00753791,Adventure,Fantasy,DJCHHD,DISNHD,WCBSDT,daytime,...,0,0,0,0,0,0,0,0,0,3
2,\x2e458bac147825dee7ecc2e12b98c7dbdbcb7d33,EP01324002,EP02306553,SH00765918,News,Talk,CNNHD,AMCHD,TBSHD,daytime,...,0,0,0,0,0,0,0,0,0,3
3,\x12cda956ba686fa51d9b60f382eafb9cebb090fc,EP01422884,EP02363888,EP00001893,Talk,Reality,FNCHD,CNNHD,SPIKEHD,primetime,...,0,0,0,0,0,0,0,0,0,3
4,\x185a49e9c8b2b7341ce9d07651707e7482376597,EP00660961,EP01448290,EP01245870,Reality,Sitcom,DISNHD,AETVHD,NIKHD,overnight,...,0,0,0,0,0,0,0,0,0,3


In [26]:
first11= analyze_me.iloc[:,:12]
analyze_removed_shows = pd.concat([first11,analyze_me.iloc[:,-1]], axis=1)

In [27]:
analyze_removed_shows.head()

,sdid,show1,show2,show3,genre1,genre2,callsign1,callsign2,callsign3,daypart1,daypart2,dma,cluster
0,\x28d8badcfd0a79edcee94962ee23fbfa1e09a589,SH00019917,EP02215786,SH01092572,Sports Related,Reality,ESPNHD,NFLHD,NBATV,latefringe,overnight,839,0
1,\x05a58f5e3d66fa67c1d713ac6826522b0f059d19,EP01532516,EP01333672,EP00753791,Adventure,Fantasy,DJCHHD,DISNHD,WCBSDT,daytime,earlymorning,501,3
2,\x2e458bac147825dee7ecc2e12b98c7dbdbcb7d33,EP01324002,EP02306553,SH00765918,News,Talk,CNNHD,AMCHD,TBSHD,daytime,primetime,789,3
3,\x12cda956ba686fa51d9b60f382eafb9cebb090fc,EP01422884,EP02363888,EP00001893,Talk,Reality,FNCHD,CNNHD,SPIKEHD,primetime,earlyfringe,627,3
4,\x185a49e9c8b2b7341ce9d07651707e7482376597,EP00660961,EP01448290,EP01245870,Reality,Sitcom,DISNHD,AETVHD,NIKHD,overnight,earlymorning,534,3


In [47]:
analyze_removed_shows.iloc[:,1:].groupby(['cluster']).agg(lambda x:x.value_counts().index[0])

,show1,show2,show3,genre1,genre2,callsign1,callsign2,callsign3,daypart1,daypart2,dma
cluster,,,,,,,,,,,
0,SH00019917,SH00019917,SH00019917,Sports Related,Sports Related,ESPNHD,ESPNHD,ESPNHD,primetime,primetime,501
1,EP00552080,EP01567915,EP01567915,Reality,Drama,WNBCDT,WNBCDT,WNBCDT,primetime,daytime,501
2,SH02330741,SH02330741,SH02330741,Reality,Reality,WCBSDT,WCBSDT,WCBSDT,primetime,daytime,501
3,EP00552080,EP02306553,EP00552080,Reality,Reality,HGTVD,WABCDT,WABCDT,primetime,daytime,501
4,EP01324002,EP01324002,EP01324002,Reality,Drama,AMCHD,AMCHD,AMCHD,primetime,daytime,501


In [48]:
analyze_removed_shows.to_csv('group_it_up.csv')